Build a regression model.

In [135]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import linear_model, datasets

In [136]:
EDA_bikestations = pd.read_csv('/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Data_Statistical_Modeling/Statistical_Modeling_Project/Statistical-Modelling-Project./data/EDA_bikestations.csv')

EDA_bikestations

,station_id,free bikes,empty slots,total bike slots,average rating,total ratings,average popularity,restaurant_count,bar_count,cafe_counts,bakery_counts,coffee_counts
0,0,0,32,32,7.585366,2772.0,0.823715,18,1,8,2,2
1,1,0,23,23,7.744444,1808.0,0.750702,18,1,6,0,2
2,2,7,26,33,7.354545,1134.0,0.807498,18,0,4,4,1
3,3,8,6,14,7.709375,1669.0,0.807738,12,1,4,2,3
4,4,6,17,23,7.516000,6061.0,0.777604,21,3,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
190,190,12,18,30,7.988000,7211.0,0.872005,17,4,3,3,1
191,191,1,40,41,7.706250,774.0,0.730178,13,0,1,1,2
192,192,0,17,17,7.446341,3428.0,0.769418,24,3,3,2,1
193,193,16,4,20,8.192000,13673.0,0.940997,17,9,2,2,2


In [153]:

X = EDA_bikestations.drop(columns=['station_id','free bikes','empty slots','total bike slots'])

y_totalbikes = EDA_bikestations['total bike slots']
y_freebikes = EDA_bikestations['free bikes']

## Pre-Model statistical testing

Building upon the pairplot and scatterplots from **joining_data.ipynb**, we shall test for regression model assumptions prior to final analysis

- Linear Relationships
    - Pearson's Correlation Coeffiecent
- Multicolinarity
    - Variance Inflation Factors (VIF)

## Linear Correlation Coefficients



In [154]:
# Linear Relationship | Pearson's Correlation

from scipy.stats import pearsonr

correlation_results = {
    'feature': [],
    'correlation': [],
    'p_value': []
}

print('Total Bike Slots Linear Correlations')
print('')
for variable in X.columns:
    corr, p = pearsonr(X[variable], y_totalbikes)
    correlation_results['feature'].append(variable)
    correlation_results['correlation'].append(round(corr,3))
    correlation_results['p_value'].append(round(p,3))

Total_Bikes_Correlation = pd.DataFrame(correlation_results)

Total_Bikes_Correlation['Statistically Significant'] = [
    True if p < 0.05 
    else False 
    for p in Total_Bikes_Correlation['p_value']
]

Total_Bikes_Correlation['Correlation Strength'] = [
    'Weak' if -0.5 < value < 0.5
    else 'Strong'
    for value in Total_Bikes_Correlation['correlation']
]

Total_Bikes_Correlation
    

Total Bike Slots Linear Correlations



,feature,correlation,p_value,Statistically Significant,Correlation Strength
0,average rating,-0.072,0.316,False,Weak
1,total ratings,-0.022,0.758,False,Weak
2,average popularity,-0.149,0.038,True,Weak
3,restaurant_count,-0.071,0.322,False,Weak
4,bar_count,0.037,0.612,False,Weak
5,cafe_counts,-0.028,0.697,False,Weak
6,bakery_counts,-0.211,0.003,True,Weak
7,coffee_counts,0.044,0.538,False,Weak


In [155]:
correlation_results = {
    'feature': [],
    'correlation': [],
    'p_value': []
}

print('Free Bikes Linear Correlations')
print('')
for variable in X.columns:
    corr, p = pearsonr(X[variable], y_freebikes)
    correlation_results['feature'].append(variable)
    correlation_results['correlation'].append(round(corr,3))
    correlation_results['p_value'].append(round(p,3))

Free_Bikes_Correlation = pd.DataFrame(correlation_results)

Free_Bikes_Correlation['Statistically Significant'] = [
    True if p < 0.05 
    else False 
    for p in Free_Bikes_Correlation['p_value']
]

Free_Bikes_Correlation['Correlation Strength'] = [
    'Weak' if -0.5 < value < 0.5
    else 'Strong' 
    for value in Free_Bikes_Correlation['correlation']
]

Free_Bikes_Correlation


Free Bikes Linear Correlations



,feature,correlation,p_value,Statistically Significant,Correlation Strength
0,average rating,0.231,0.001,True,Weak
1,total ratings,0.342,0.000,True,Weak
2,average popularity,0.134,0.062,False,Weak
3,restaurant_count,-0.016,0.827,False,Weak
4,bar_count,0.462,0.000,True,Weak
5,cafe_counts,-0.199,0.005,True,Weak
6,bakery_counts,-0.087,0.225,False,Weak
7,coffee_counts,0.055,0.442,False,Weak


## Linear Correlation Coefficients Results

Pearsonr Correlation Coefficient Tests 
- All X-variables against two y-variables (Total Bike Slots & Free Bikes) 
    - All results showed weak linear correlations. 

- Most p-values insignificant, with results above predetermined signifiance level alpha of 5% 

## Multicollinearity 

Testing for Multicollinearity with statistical testing | **Variance Inflation Factors (VIF)**

In [156]:
# AI Assistance

# No Multicollinearity | Variance Inflation Factors (VIF)

# Utilise Assumption Threshold of 5

from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif['Multicollinearity'] = [ True if value >= 5 
                            else False
                            for value in vif['VIF']
]
vif

,feature,VIF,Multicollinearity
0,average rating,315.042105,True
1,total ratings,5.166563,True
2,average popularity,371.931468,True
3,restaurant_count,15.283779,True
4,bar_count,3.408861,False
5,cafe_counts,6.510545,True
6,bakery_counts,3.617797,False
7,coffee_counts,2.532380,False


In [ ]:
# Test Multicollinearity between highest variables: Ratings, Popularity & Restaurant Count

vif = pd.DataFrame()
vif["feature"] = X[['average rating','average popularity','restaurant_count']].columns
vif["VIF"] = [variance_inflation_factor(X[['average rating','average popularity','restaurant_count']].values, i) for i in range(X[['average rating','average popularity','restaurant_count']].shape[1])]

vif['Multicollinearity'] = [ True if value >= 5 
                            else False
                            for value in vif['VIF']
]

vif

,feature,VIF,Multicollinearity
0,average rating,284.041301,True
1,average popularity,300.188472,True
2,restaurant_count,13.733293,True


In [159]:
# Drop ratings to view Multicollinearity among category counts

vif_categories = pd.DataFrame()
X_categories = X.drop(columns=['average rating','total ratings','average popularity'])

vif_categories["feature"] = X_categories.columns
vif_categories["VIF"] = [variance_inflation_factor(X_categories.values, i) for i in range(X_categories.shape[1])]

vif_categories['Multicollinearity'] = [ True if value >= 5 
                            else False
                            for value in vif_categories['VIF']
]
vif_categories

,feature,VIF,Multicollinearity
0,restaurant_count,5.806049,True
1,bar_count,1.958126,False
2,cafe_counts,3.562279,False
3,bakery_counts,3.365277,False
4,coffee_counts,2.304633,False


## Multicollinearity Results

Extremely strong multicollinearity present among 'average rating' and 'popularity.

Secondly, very strong multicollinearity among Rating & Popularity variables with Restaurant Counts.

Seperating of variables of 'Category Types' and 'Ratings'.
- Acknowledge remaining high multicollinearity value for 'Restaurant_count" at 5.79 and impact on Regression Analysis

# Regression Models
Provide model output and an interpretation of the results. 

## Multivariate Regression Models


In [160]:
# Location Categories analysis against Total Bike Slots

X_categories = sm.add_constant(X_categories)
lin_reg = sm.OLS(y_totalbikes,X_categories)

model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:       total bike slots   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     2.173
Date:                Sun, 27 Jul 2025   Prob (F-statistic):             0.0588
Time:                        22:05:27   Log-Likelihood:                -668.15
No. Observations:                 195   AIC:                             1348.
Df Residuals:                     189   BIC:                             1368.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               23.4155      2.754  

In [144]:
# Location Categories analysis against Free Bikes

X_categories = sm.add_constant(X_categories)
lin_reg = sm.OLS(y_freebikes,X_categories)

model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:             free bikes   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.223
Method:                 Least Squares   F-statistic:                     10.26
Date:                Sun, 27 Jul 2025   Prob (F-statistic):           8.16e-10
Time:                        21:58:46   Log-Likelihood:                -591.12
No. Observations:                 195   AIC:                             1196.
Df Residuals:                     188   BIC:                             1219.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  7.2590      5

### Multivariate Regression Models Analysis

- Location Categories Vs Total Bike Slots
    - Adjusted R-squared / Coefficient Of Determination = 0.029
        - Rounded 3% of variation in Total Bike Slotsavailable across Lisbon, Portugal can be explained by this model.
        - Locations Categories (e.g Restaurant, Bar, Cafe, Bakeries, Coffee) is a **poor** explanatory variable for Total Bike Slots at CityBike stations in Lisbon


- Location Categories Vs Free Bikes
    - R-squared / Coefficient Of Determination = 0.228
        - Rounded 23% of variation in Free Bikes available across Lisbon, Portgual can be explain by this model.
        - Location Cateogires (e.g Restaurant, Bar, Cafe, Bakeries, Coffee) is a **moderate** explnatory variable for Free bikes at CityBike stations in Lisbon, at the relevant time period of **12:39 AM Sunday, July 27 2025.**

    - Variables
        - Bars
            - Coeffieicent
                - Holding all other features constant, for each additional 'bar' location surrounding CityBike Stations, the number of free bikes available is estiamted to **increase by 0.956**.
            - p-value
                - At 0.00, highly statistically significant 
        - Cafes
            - Coeffieicent
                - Holding all other features constant, for each additional 'cafe' location surrounding CityBike Stations, the number of free bikes available is estiamted to **decrease by 0.485**.
            - p-value
                - At 0.12, highly statistically significant. Meeting our alpha signifiance level of 5%

    - Further Regression Model Analysis

In [145]:
# Categories Bars & Cafes analysis against Free Bikes

X_selected = sm.add_constant(X_categories[['bar_count','cafe_counts']])
lin_reg = sm.OLS(y_freebikes,X_selected)

model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:             free bikes   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.226
Method:                 Least Squares   F-statistic:                     29.40
Date:                Sun, 27 Jul 2025   Prob (F-statistic):           7.30e-12
Time:                        21:58:46   Log-Likelihood:                -592.68
No. Observations:                 195   AIC:                             1191.
Df Residuals:                     192   BIC:                             1201.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           4.8611      0.950      5.117      

- Inference
    - Based on our two strongest variables (Bars & Cafes), we can infer that 23% of variation in Free Bikes across Lisbon can be explain by these two predictors.

    **TimeStamp** = **12:39 AM Sunday, July 27 2025.**

    - Bars
        - Each bar within a 1,000 metre radius of a CityBike station increases 0.91 Free Bikes.
            - Inference can be customers ride bikes to stations with local bars during this TimeStamp
            OR
            - CityBike allocates 0.91 more bikes at stations at this Timestamp for anticaption of customer use
    
    - Cafes
        - Each cafe within a 1,000 metre radius of a CityBike station decreases 0.40 Free Bikes
            - Inference can be CityBike Stations with cafes have lower quantity of customers riding bikes during this TimeStamp
            OR
            - CityBike allocates 0.40 less bikes at stations durign this TimeStamp


    

## Linear Regression Models

Compare 'Average Popularity' of local locations with y-varibles


In [161]:

X_pop = EDA_bikestations['average popularity']
y_freebikes
y_totalbikes

# Total Bike Slots
X_pop = sm.add_constant(X_pop)
lin_reg = sm.OLS(y_totalbikes,X_pop)

model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:       total bike slots   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     4.387
Date:                Sun, 27 Jul 2025   Prob (F-statistic):             0.0375
Time:                        22:08:13   Log-Likelihood:                -671.41
No. Observations:                 195   AIC:                             1347.
Df Residuals:                     193   BIC:                             1353.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 36.3187      7

In [162]:
# Free Bike Slots
X_pop = sm.add_constant(X_pop)
lin_reg = sm.OLS(y_freebikes,X_pop)

model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:             free bikes   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     3.516
Date:                Sun, 27 Jul 2025   Prob (F-statistic):             0.0623
Time:                        22:08:40   Log-Likelihood:                -616.96
No. Observations:                 195   AIC:                             1238.
Df Residuals:                     193   BIC:                             1244.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -5.4918      5

## Linear Regression Models Analysis

Comparing 'Average Popularity' of local locations with Free Bikes & Total Bike Slots

- As visualised in 'joining_data.ipynb' pairplots and scatterplots and the above 'Pearson R' tests, the 'Average Popularity' does not have strong linear relationships with either y-variable.

- 'Average Popularity' of locations within 1,000 metre radius of CityBike Stations does have a strong correlation coefficient in a Linear Regression of 12.26. However, with a p-value of 0.088, marking it **statistically insignificant**, the expected variabaility of 'Free Bikes' can only be explained by 'Average Popularity' to 1%

- Similarly, 'Average Popularity' has a strong negative correlation coefficient in linear regression with 'Total Bike Slots' at -18.04. However, with a p-value of 0.058, marking it **statistically insignificant**, and expected variabaility of 'Total Bike Slot'  only explained by 'Average Popularity' to 1.3%.

‘Average Popularity is an **extremely poor predictor** of both Free Bikes and Total Bike Slots at CityBike stations, with low explanatory power and no statistically significant relationship observed in either regression model.

# Stretch

How can you turn the regression model into a classification model?